# Exploratory Data Analysis of Epicurious Scrape in a JSON file

This is an idealized workflow for Aaron Chen in looking at data science problems. It likely isn't the best path, nor has he rigidly applied or stuck to this ideal, but he wishes that he worked this way more frequently.

## Purpose: Work through some exploratory data analysis of the Epicurious scrape on stream. Try to write some functions to help process the data.

### Author: Aaron Chen


---

### If needed, run shell commands here

In [1]:
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.9/13.9 MB 36.4 MB/s eta 0:00:0000:0100:01
You should consider upgrading via the '/home/awchen/Repos/Projects/MeaLeon/.venv/bin/python -m pip install --upgrade pip' command.
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


---

## External Resources

List out references or documentation that has helped you with this notebook

### Code
Regex Checker: https://regex101.com/

#### Scikit-learn
1. https://scikit-learn.org/stable/modules/decomposition.html#latent-dirichlet-allocation-lda
2. 

### Data

For this notebook, the data is stored in the repo base folder/data/raw

### Process

Are there steps or tutorials you are following? Those are things I try to list in Process

___

## Import necessary libraries

In [2]:
# from collections import defaultdict 
# import dvc.api
# import gensim
# import gensim.corpora as corpora
# from gensim.models import CoherenceModel, Phrases
# from gensim.utils import simple_preprocess
# import logging
# import mlflow
# import mlflow.sklearn
# import mlflow.spacy
import numpy as np
import pandas as pd
# from pprint import pprint
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

import spacy
import en_core_web_sm
from spacy.lang.en.stop_words import STOP_WORDS
# from spacy.lemmatizer import Lemmatizer
from tqdm import tqdm
# from typing import List
import unicodedata

---

## Define helper functions

My workflow is to try things with code cells, then when the code cells get messy and repetitive, to convert into helper functions that can be called.

When the helper functions are getting used a lot, it is usually better to convert them to scripts or classes that can be called/instantiated

In [3]:
# def remove_empties(deficiency_text: List) -> List:
#     """This function takes in a list of strings and removes empty strings from the list. The function is needed 
#     because if the list does not contain empty strings, the default remove() function returns None and an Error."""
    
#     filtered = list(filter(lambda x: x != '', deficiency_text))

#     return filtered

In [4]:
# def lemmatizer(doc):
#     # This takes in a doc of tokens from the NER and lemmatizes them. 
#     # Pronouns (like "I" and "you" get lemmatized to '-PRON-', so I'm removing those.
#     doc = [token.lemma_ for token in doc if token.lemma_ != '-PRON-']
#     doc = u' '.join(doc)
#     return nlp.make_doc(doc)

In [5]:
# def remove_stopwords(doc):
#     # This will remove stopwords and punctuation.
#     # Use token.text to return strings, which we'll need for Gensim.
#     doc = [token for token in doc if token.is_stop != True and token.is_punct != True]
#     return doc

In [6]:
# nlp.add_pipe(lemmatizer,name='lemmatizer',after='ner')
# nlp.add_pipe(remove_stopwords, name="stopwords", last=True)

### Import local script

I started grouping this in with importing libraries, but putting them at the bottom of the list

In [7]:
# import project_path
# import src.nhsn_vac_df_builder as nhsn_vac

---

## Define global variables 
### Remember to refactor these out, not ideal

In [8]:
data_path = "../../data/recipes-en-201706/epicurious-recipes_m2.json"

---

## Running Commentary

1. I used numbered lists to keep track of things I noticed

### To Do

1. Try to determine consistency of nested data structures
   1. Is the photoData or number of things inside photoData the same from record to record
   2. What about for tag?
2. How to handle nulls?
   1. Author
   2. Tag
3. Convert pubDate to actual timestamp
4. Convert ScrapeDate to actual timestamp
   
**5. Append new columns for relevant nested structures and unfold them**

6. Determine actual types of `ingredients` and `prepSteps`
7. Continue working through test example of single recipe to feed into spaCy and then sklearn.feature_extraction.text stack
8. Will need to remove numbers, punctuation

---

## Importing and viewing the data as a dataframe

In [9]:
epic_dataframe = pd.read_json(path_or_buf=data_path)

In [10]:
epic_dataframe.head()

,id,dek,hed,pubDate,author,type,url,photoData,tag,aggregateRating,ingredients,prepSteps,reviewsCount,willMakeAgainPct,dateCrawled
0,54a2b6b019925f464b373351,How does fried chicken achieve No. 1 status? B...,Pickle-Brined Fried Chicken,2014-08-19T04:00:00.000Z,[],recipe,/recipes/food/views/pickle-brined-fried-chicke...,"{'id': '54a2b64a6529d92b2c003409', 'filename':...","{'category': 'ingredient', 'name': 'Chicken', ...",3.11,"[1 tablespoons yellow mustard seeds, 1 tablesp...",[Toast mustard and coriander seeds in a dry me...,7,100,1498547035
1,54a408a019925f464b3733bc,Spinaci all'Ebraica,Spinach Jewish Style,2008-09-09T04:00:00.000Z,[{'name': 'Edda Servi Machlin'}],recipe,/recipes/food/views/spinach-jewish-style-350152,"{'id': '56746182accb4c9831e45e0a', 'filename':...","{'category': 'cuisine', 'name': 'Italian', 'ur...",3.22,"[3 pounds small-leaved bulk spinach, Salt, 1/2...",[Remove the stems and roots from the spinach. ...,5,80,1498547740
2,54a408a26529d92b2c003631,"This majestic, moist, and richly spiced honey ...",New Year’s Honey Cake,2008-09-10T04:00:00.000Z,[{'name': 'Marcy Goldman'}],recipe,/recipes/food/views/majestic-and-moist-new-yea...,"{'id': '55e85ba4cf90d6663f728014', 'filename':...","{'category': 'cuisine', 'name': 'Jewish', 'url...",3.62,"[3 1/2 cups all-purpose flour, 1 tablespoon ba...",[I like this cake best baked in a 9-inch angel...,105,88,1498547738
3,54a408a66529d92b2c003638,The idea for this sandwich came to me when my ...,The B.L.A.Bagel with Lox and Avocado,2008-09-08T04:00:00.000Z,[{'name': 'Faye Levy'}],recipe,/recipes/food/views/the-b-l-a-bagel-with-lox-a...,"{'id': '5674617e47d1a28026045e4f', 'filename':...","{'category': 'cuisine', 'name': 'Jewish', 'url...",4.00,"[1 small ripe avocado, preferably Hass (see No...","[A short time before serving, mash avocado and...",7,100,1498547740
4,54a408a719925f464b3733cc,"In 1930, Simon Agranat, the chief justice of t...",Shakshuka a la Doktor Shakshuka,2008-09-09T04:00:00.000Z,[{'name': 'Joan Nathan'}],recipe,/recipes/food/views/shakshuka-a-la-doktor-shak...,"{'id': '56746183b47c050a284a4e15', 'filename':...","{'category': 'cuisine', 'name': 'Jewish', 'url...",2.71,"[2 pounds fresh tomatoes, unpeeled and cut in ...","[1. Place the tomatoes, garlic, salt, paprika,...",7,83,1498547740


In [11]:
epic_dataframe['type'].value_counts()

recipe    34756
Name: type, dtype: int64

In [12]:
epic_dataframe.shape

(34756, 15)

In [13]:
epic_dataframe.describe()

,aggregateRating,reviewsCount,willMakeAgainPct,dateCrawled
count,34756.000000,34756.000000,34756.000000,3.475600e+04
mean,2.937041,19.467257,75.435579,1.498551e+09
std,1.079268,36.121799,31.064887,2.276313e+04
min,0.000000,0.000000,0.000000,1.498546e+09
25%,2.860000,3.000000,70.000000,1.498548e+09
50%,3.270000,9.000000,87.000000,1.498548e+09
75%,3.580000,22.000000,98.000000,1.498549e+09
max,4.000000,1586.000000,100.000000,1.498869e+09


In [14]:
epic_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34756 entries, 0 to 34755
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                34756 non-null  object 
 1   dek               34756 non-null  object 
 2   hed               34756 non-null  object 
 3   pubDate           34756 non-null  object 
 4   author            34756 non-null  object 
 5   type              34756 non-null  object 
 6   url               34756 non-null  object 
 7   photoData         34756 non-null  object 
 8   tag               34655 non-null  object 
 9   aggregateRating   34756 non-null  float64
 10  ingredients       34656 non-null  object 
 11  prepSteps         34756 non-null  object 
 12  reviewsCount      34756 non-null  int64  
 13  willMakeAgainPct  34756 non-null  int64  
 14  dateCrawled       34756 non-null  int64  
dtypes: float64(1), int64(3), object(11)
memory usage: 4.0+ MB


In [15]:
epic_dataframe['aggregateRating'].value_counts()

0.00    3316
3.00    3113
4.00    2161
3.50    1286
3.33     894
        ... 
1.38       1
1.62       1
1.48       1
2.51       1
1.55       1
Name: aggregateRating, Length: 234, dtype: int64

Columns:

    Index

    ID: string

    dek: appears to be description of the recipe, string

    hed: Appears to be title, string

    pubDate: appears to be publication date, may need to reformat to datetime objects

    author: appears that each record contains an array (list), inside each list is a dictionary with 'name' as the key and author name as the value. Notably, not a unique identifier for the value. Because the data is technically nested, may need to extract and transform and add columns to dataframe

    type: string, but all the values are exactly the same and they are all in the category of "recipe". Drop column

    url: Appears to be a long string leading to where the recipe can be found on Epicurious's website

    photoData: nested structure, inside each record is a dictionary

    tag: each record contains a dictionary. may need to extract and transform and add columns to dataframe

    aggregateRating: float, let's say it's out of 4.0

    ingredients: appears to be a list, does look like a list of strings

    prepSteps: appears to be a list, does look like a list of strings

    reviewsCount: int

    willMakeAgainPct: integer	
    
    dateCrawled: appears to be a unix timestamp

Let's take a look at the possibly problematic columns and see if the data structures make sense or how we can approach transforming them into new columns for the dataset

In [16]:
epic_dataframe.loc[0]

id                                           54a2b6b019925f464b373351
dek                 How does fried chicken achieve No. 1 status? B...
hed                                       Pickle-Brined Fried Chicken
pubDate                                      2014-08-19T04:00:00.000Z
author                                                             []
type                                                           recipe
url                 /recipes/food/views/pickle-brined-fried-chicke...
photoData           {'id': '54a2b64a6529d92b2c003409', 'filename':...
tag                 {'category': 'ingredient', 'name': 'Chicken', ...
aggregateRating                                                  3.11
ingredients         [1 tablespoons yellow mustard seeds, 1 tablesp...
prepSteps           [Toast mustard and coriander seeds in a dry me...
reviewsCount                                                        7
willMakeAgainPct                                                  100
dateCrawled         

In [17]:
epic_dataframe.loc[0]["photoData"]

{'id': '54a2b64a6529d92b2c003409',
 'filename': '51247610_fried-chicken_1x1.jpg',
 'caption': 'Pickle-Brined Fried Chicken',
 'credit': 'Michael Graydon and Nikole Herriott',
 'promoTitle': 'Pickle-Brined Fried Chicken',
 'title': 'Pickle-Brined Fried Chicken',
 'orientation': 'landscape',
 'restrictCropping': False}

It looks like photoData contains:
    1. photo ID, string
    2. photo filename, string
    3. photo caption, string
    4. photo credit, string
    5. promoTitle, string
    6. title, string
       1. caption, promoTitle, and title could be all the same
    7. orientation, string
    8. restrictCropping: boolean

Of these, maybe we should keep
id => photoID
filename => photoFilename
caption => photoCaption
credit => photoCredit


In [18]:
epic_dataframe.loc[0]["tag"]

{'category': 'ingredient',
 'name': 'Chicken',
 'url': '',
 'photosBadgeAltText': '',
 'photosBadgeFileName': '',
 'photosBadgeID': '',
 'photosBadgeRelatedUri': ''}

In [19]:
epic_dataframe.loc[100]["tag"]

{'category': 'ingredient',
 'name': 'Champagne',
 'url': '',
 'photosBadgeAltText': '',
 'photosBadgeFileName': '',
 'photosBadgeID': '',
 'photosBadgeRelatedUri': ''}

In [20]:
epic_dataframe.loc[10]["tag"]

{'category': 'type',
 'name': 'Cake',
 'url': '',
 'photosBadgeAltText': '',
 'photosBadgeFileName': '',
 'photosBadgeID': '',
 'photosBadgeRelatedUri': ''}

In [21]:
epic_dataframe.loc[1]["tag"]

{'category': 'cuisine',
 'name': 'Italian',
 'url': '',
 'photosBadgeAltText': '',
 'photosBadgeFileName': '',
 'photosBadgeID': '',
 'photosBadgeRelatedUri': ''}

In [22]:
epic_dataframe.loc[1]["ingredients"]

['3 pounds small-leaved bulk spinach',
 'Salt',
 '1/2 cup dark seedless raisins',
 '1 cup lukewarm water',
 '6 tablespoons olive oil',
 '1/2 small onion, minced',
 '1/4 cup pignoli (pine nuts)',
 'Freshly ground black pepper',
 'Dash nutmeg']

In [23]:
type(epic_dataframe.loc[1]["ingredients"])

list

In [24]:
epic_dataframe.loc[1]["dek"]

"Spinaci all'Ebraica"

### Let's skip ahead and throw this into CountVectorizer

In [25]:
all_recipes_list = epic_dataframe['ingredients'].str.join(" ")
# .apply(" ".join).str.lower()
#print(type(all_recipes_list))
all_recipes_list
# for i in range(0,5):
#     print((i, all_recipes_list[i]))

0        1 tablespoons yellow mustard seeds 1 tablespoo...
1        3 pounds small-leaved bulk spinach Salt 1/2 cu...
2        3 1/2 cups all-purpose flour 1 tablespoon baki...
3        1 small ripe avocado, preferably Hass (see Not...
4        2 pounds fresh tomatoes, unpeeled and cut in q...
                               ...                        
34751    1 tablespoon unsalted butter, at room temperat...
34752    8 tablespoons (1 stick) salted butter, at room...
34753    3 tablespoons unsalted butter, plus more for g...
34754    Coarse salt 2 lime wedges 2 ounces tomato juic...
34755    1 bottle (375 ml) sour beer, such as Almanac C...
Name: ingredients, Length: 34756, dtype: object

In [26]:
nlp = spacy.load("en_core_web_sm")

In [33]:
first_rec = all_recipes_list[0]
print(first_rec)

1 tablespoons yellow mustard seeds 1 tablespoons brown mustard seeds 1 1/2 teaspoons coriander seeds 1 cup apple cider vinegar 2/3 cup kosher salt 1/3 cup sugar 1/4 cup chopped fresh dill 8 skinless, boneless chicken thighs (about 3 pounds), halved, quartered if large Vegetable oil (for frying; about 10 cups) 2 cups buttermilk 2 cups all-purpose flour Kosher salt Honey, flaky sea salt (such as Maldon), toasted benne or sesame seeds, hot sauce (for serving) A deep-fry thermometer


In [ ]:
doc_first_rec = nlp(first_rec)
for token in doc_first_rec:
    if token.like_num == False:
        print(token.text, token.pos_, token.ent_type_, token.lemma_, token.is_digit)
    else:
        continue

In [57]:
epic_dataframe['tag'][2]

{'category': 'cuisine',
 'name': 'Jewish',
 'url': '',
 'photosBadgeAltText': '',
 'photosBadgeFileName': '',
 'photosBadgeID': '',
 'photosBadgeRelatedUri': ''}

In [70]:
epic_dataframe['cuisine_name'] = epic_dataframe['tag'].apply(lambda x: x['name'] if not pd.isna(x) and x['category'] == 'cuisine' else 'Cuisine Missing')

In [71]:
epic_dataframe['cuisine_name'].head()

0    Cuisine Missing
1            Italian
2             Jewish
3             Jewish
4             Jewish
Name: cuisine_name, dtype: object

In [68]:
epic_dataframe['cuisine_name'][0]

'Missing'

In [69]:
print(epic_dataframe.shape)
print(epic_dataframe[epic_dataframe['cuisine_name'] != 'Missing'].shape)


(34756, 16)
(14847, 16)


In [72]:
epic_dataframe[epic_dataframe['cuisine_name'] == 'Cuisine Missing']

,id,dek,hed,pubDate,author,type,url,photoData,tag,aggregateRating,ingredients,prepSteps,reviewsCount,willMakeAgainPct,dateCrawled,cuisine_name
0,54a2b6b019925f464b373351,How does fried chicken achieve No. 1 status? B...,Pickle-Brined Fried Chicken,2014-08-19T04:00:00.000Z,[],recipe,/recipes/food/views/pickle-brined-fried-chicke...,"{'id': '54a2b64a6529d92b2c003409', 'filename':...","{'category': 'ingredient', 'name': 'Chicken', ...",3.11,"[1 tablespoons yellow mustard seeds, 1 tablesp...",[Toast mustard and coriander seeds in a dry me...,7,100,1498547035,Cuisine Missing
5,54a408a919925f464b3733d3,Although Nelly Custis omitted sugar in her rec...,Rice Pancakes,2012-02-17T04:00:00.000Z,[{'name': 'Stephen A. McLeod'}],recipe,/recipes/food/views/rice-pancakes-394729,"{'id': '56746183b47c050a284a4e15', 'filename':...","{'category': 'ingredient', 'name': 'Milk/Cream...",0.00,"[1 1/2 cups cooked rice, 2 cups heavy cream, 2...","[1. Combine the rice, cream, and butter. Add t...",0,0,1498547293,Cuisine Missing
6,54a408aa19925f464b3733d6,Editor's note: This recipe is adapted with per...,Jack-O'-Lantern,2008-09-09T04:00:00.000Z,[{'name': 'Matthew Mead'}],recipe,/recipes/food/views/jack-o-lantern-350068,"{'id': '560d7907f9a841923089d7da', 'filename':...","{'category': 'type', 'name': 'Cake', 'url': ''...",1.00,"[2 tablespoons shortening, 2 tablespoons flour...",[1. Preheat the oven to 350°F. Lightly grease ...,1,0,1498547740,Cuisine Missing
7,54a408ab19925f464b3733da,Editor's note: This recipe is reprinted with p...,Seven-Minute Frosting,2008-09-09T04:00:00.000Z,[{'name': 'Matthew Mead'}],recipe,/recipes/food/views/seven-minute-frosting-350069,"{'id': '5674617eb47c050a284a4e11', 'filename':...","{'category': 'equipment', 'name': 'Mixer', 'ur...",3.53,"[1 1/2 cups sugar, 1/3 cup cold water, 2 egg w...","[1. Combine the sugar, water, egg whites, and ...",8,75,1498547740,Cuisine Missing
8,54a408ac19925f464b3733de,Editor's note: This recipe is reprinted with p...,Creamy White Frosting,2008-09-09T04:00:00.000Z,[{'name': 'Matthew Mead'}],recipe,/recipes/food/views/creamy-white-frosting-350079,"{'id': '5674617e47d1a28026045e4f', 'filename':...","{'category': 'equipment', 'name': 'Mixer', 'ur...",2.00,"[1 cup vegetable shortening, 1 1/2 teaspoons v...","[1. With a mixer on medium speed, beat togethe...",5,0,1498547740,Cuisine Missing
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34751,59541a31bff3052847ae2107,Buttering the bread before you waffle it ensur...,Waffled Ham and Cheese Melt with Maple Butter,2017-06-29T14:59:01.368Z,[{'name': 'Daniel Shumski'}],recipe,/recipes/food/views/waffled-ham-and-cheese-mel...,"{'id': '595420c2d52ca90dc28200e6', 'filename':...","{'category': 'tag', 'name': 'Small Plates', 'u...",0.00,"[1 tablespoon unsalted butter, at room tempera...","[Preheat the waffle iron on low., Spread a thi...",0,0,1498857706,Cuisine Missing
34752,5954233ad52ca90dc28200e7,"Spread this easy compound butter on waffles, p...",Maple Butter,2017-06-01T14:57:00.000Z,[{'name': 'Daniel Shumski'}],recipe,/recipes/food/views/maple-butter,"{'id': '5674617eb47c050a284a4e11', 'filename':...","{'category': 'meal', 'name': 'Breakfast', 'url...",0.00,"[8 tablespoons (1 stick) salted butter, at roo...",[Combine the ingredients in a medium-size bowl...,0,0,1498857726,Cuisine Missing
34753,595424c2109c972493636f83,Leftover mac and cheese is not exactly one of ...,Waffled Macaroni and Cheese,2017-06-29T14:54:24.234Z,[{'name': 'Daniel Shumski'}],recipe,/recipes/food/views/waffled-macaroni-and-cheese,"{'id': '5954259cd83a053f4d33bc74', 'filename':...","{'category': 'tag', 'name': 'Small Plates', 'u...",0.00,"[3 tablespoons unsalted butter, plus more for ...",[Preheat the oven to 375°F. Butter a 9x5-inch ...,0,0,1498857706,Cuisine Missing
34754,5956638625dc3d1d829b7166,A classic Mexican beer cocktail you can sip al...,Classic Michelada,2017-06-15T16:41:00.000Z,[{'name': 'Kat Odell'}],recipe,/recipes/food/views/classic-michelada,"{'id': '

In [ ]:
for word in test_rec_list:     
    print(unicodedata.normalize("NFKD", word))

In [28]:
cv = CountVectorizer(input=aa)

NameError: name 'aa' is not defined